In [1]:
uv add pandas

Note: you may need to restart the kernel to use updated packages.


Resolved 22 packages in 1ms
Audited 21 packages in 0.04ms


In [2]:
uv add plotly

Note: you may need to restart the kernel to use updated packages.


Resolved 24 packages in 725ms
Prepared 2 packages in 10.40s
Installed 2 packages in 2.75s
 + narwhals==1.42.1
 + plotly==6.1.2


In [4]:
import pandas as pd

In [ ]:
df=pd.read_pickle('.\conversations.pkl')

In [ ]:
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objects as go

# 1) Enable offline
init_notebook_mode(connected=True)

# 2) Reset index and sort
df_reset  = df.reset_index()
df_sorted = df_reset.sort_values(['airline','conversation','tweet'])

# 3) Filter to the airline you confirmed exists
AIRLINE_ID = '18332190'   # ← replace with a valid ID from the printout
df_airline = df_sorted[df_sorted['airline'] == AIRLINE_ID]

if df_airline.empty:
    print(f"No data found for airline {AIRLINE_ID}")
else:
    # 4) Compute first & last sentiments
    trans = (
        df_airline
        .groupby(['airline','conversation'])['sentiment_label']
        .agg(first='first', last='last')
        .reset_index()
    )

    # 5) Count transitions
    counts = trans.groupby(['first','last']).size().reset_index(name='count')

    # 6) Sankey boilerplate (same as before)
    # labels = ["Positive\n", "Neutral\n", "Negative\n",
    #           "Positive\n",  "Neutral\n",  "Negative\n"]
    colors = ["#4CAF50","#9E9E9E","#F44336"]*2
    first_map = {"positive":0,"neutral":1,"negative":2}
    last_map  = {"positive":3,"neutral":4,"negative":5}
    sources = counts['first'].map(first_map)
    targets = counts['last'].map(last_map)
    values  = counts['count']
    link_colors = [colors[s] for s in sources]

    x = [0,0,0,  1,1,1]
    y = [0.9,0.5,0.1,  0.9,0.5,0.1]

    fig = go.Figure(go.Sankey(
        arrangement="fixed",
        node=dict( color=colors, x=x, y=y,
                  pad=20, thickness=30, line=dict(color="black",width=1)),
        link=dict(source=sources, target=targets, value=values, color=link_colors)
    ))

    fig.update_layout(
        title_text=f"Sentiment Transition for British Airways",
        width=600, height=600,
        margin=dict(l=50,r=50,t=80,b=50),
        plot_bgcolor="white"
    )
    iplot(fig)

In [ ]:
# data which was manually selected from the interactive plot
# tweets were sentiments improved => 15.77 K => 37.9%
# tweets were sentiments worsened => 6.14 K => 14.7%
# sentiment didnt change => 19.74 K => 47.4%
#total conversations => 41.65
